In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
from collections    import defaultdict


order_data = pd.read_csv("./data/task1_data.txt", encoding='cp949')

data_x = order_data
data_y = order_data.iloc[:, -1]


group_summary_data = []

for group in range(32):
    group_data = order_data[order_data['group'] == group]
    
    color_values = sorted(group_data['color'].unique())
    size_values = sorted(group_data['size'].unique())
    
    color_count = len(color_values)
    size_count = len(size_values)
    
    group_summary_data.append({'Group': group, 'Color Values': color_values, 'Color Count': color_count,
                               'Size Values': size_values, 'Size Count': size_count})

group_summary = pd.DataFrame(group_summary_data)

new_data = []

for group in range(32):
    # 각 그룹에 대한 데이터 추출
    group_data = order_data[order_data['group'] == group]
    
    # 색상 및 사이즈의 값들을 추출하여 정렬
    color_values = sorted(group_data['color'].unique())
    size_values = sorted(group_data['size'].unique())
    
    # 색상 및 사이즈의 모든 조합에 대해 새로운 행 생성
    for color in color_values:
        for size in size_values:
            new_data.append({'group': group, 'color': color, 'size': size})

# 리스트를 데이터프레임으로 변환
new_df = pd.DataFrame(new_data)


new_df['indexing'] = new_df.reset_index().index

# 새로운 데이터프레임과 주어진 데이터프레임 간의 매핑 수행
merged_df = pd.merge(order_data, new_df, on=['group', 'color', 'size'], how='right')
merged_df.dropna(inplace=True)

# 모든 컬럼의 데이터 타입을 int로 변환
merged_df = merged_df.astype(int)



order_dict = defaultdict(list)
merged_df = merged_df.reset_index()

order_dict = defaultdict(list)
for i in tqdm(range(2666262)):
    order_dict[merged_df['customer'][i]].append(merged_df['indexing'][i])

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist, squareform
from scipy.spatial.distance import jaccard


data_sim = np.zeros((342038,15652))
for i in order_dict.keys():
    values = order_dict[i]
    data_sim[i][values] = 1
    
    





def compute_jaccard_similarity(sparse_matrix):
    # pdist 함수를 사용하여 쌍별 Jaccard 거리를 계산한 후, 이를 유사도로 변환
    jaccard_distances = pdist(sparse_matrix, metric='jaccard')
    jaccard_similarity = 1 - jaccard_distances  # 거리를 유사도로 변환
    return squareform(jaccard_similarity)  # 정사각형 형태로 변환


###  adj matrix 생성 342개 subgraph
for i in range(0,342):
    
    similarity_matrix = compute_jaccard_similarity(data_sim[i*1000:(i+1)*1000])   
    np.savetxt(f"./adj_{i*1000}_{(i+1)*1000}_.csv", similarity_matrix, delimiter=",")